<a href="https://colab.research.google.com/github/yuuiwqy622/unsupervised-segmentation/blob/main/unsupervised_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf sample_data

In [2]:
!git clone https://github.com/NVIDIA/pix2pixHD.git

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 50.91 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [3]:
!unzip reduced-isic.zip
!rm reduced-isic.zip

Archive:  reduced-isic.zip
   creating: reduced-isic/
   creating: reduced-isic/train_edge/
  inflating: reduced-isic/train_edge/ISIC_0000000.png  
  inflating: reduced-isic/train_edge/ISIC_0000001.png  
  inflating: reduced-isic/train_edge/ISIC_0000003.png  
  inflating: reduced-isic/train_edge/ISIC_0000006.png  
  inflating: reduced-isic/train_edge/ISIC_0000007.png  
  inflating: reduced-isic/train_edge/ISIC_0000009.png  
  inflating: reduced-isic/train_edge/ISIC_0000011.png  
  inflating: reduced-isic/train_edge/ISIC_0000013.png  
  inflating: reduced-isic/train_edge/ISIC_0000014.png  
  inflating: reduced-isic/train_edge/ISIC_0000015.png  
  inflating: reduced-isic/train_edge/ISIC_0000016.png  
  inflating: reduced-isic/train_edge/ISIC_0000018.png  
  inflating: reduced-isic/train_edge/ISIC_0000019.png  
  inflating: reduced-isic/train_edge/ISIC_0000020.png  
  inflating: reduced-isic/train_edge/ISIC_0000022.png  
  inflating: reduced-isic/train_edge/ISIC_0000023.png  
  inflating:

In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
import pathlib
import os
import sys
import time
import h5py
import random

import numpy as np
import tensorflow as tf
import keras
from keras import losses
from keras import models
import matplotlib.pyplot as plt

from skimage import color, exposure, io, img_as_float, transform, filters, morphology, measure
from sklearn import metrics

Using TensorFlow backend.


In [6]:
def load_images(filepath):    
    '''
    Loads the available images.
    '''
    data_root = pathlib.Path(os.path.join(filepath))
    image_paths = [str(path) for path in  list(data_root.glob('*'))]
    return image_paths

In [7]:
def preprocess_images(image_paths, savedir, load_prev=False):
    '''
    Preprocessing for ISIC images
    '''
    if not load_prev or not os.path.isdir(savedir):
        os.makedirs(savedir, exist_ok=True)
        for path in image_paths:
            savepath = os.path.join(savedir, os.path.basename(path))
            print(path)
            im = img_as_float(io.imread(path))        
            im = transform.resize(im, (256, 256), mode="reflect", anti_aliasing=True)
            im = np.uint8(im * 255)
            io.imsave(fname=savepath,arr=im)

In [8]:
preprocess_images(load_images('reduced-isic/train_img'), 'train_img', load_prev=False)

reduced-isic/train_img/ISIC_0000021.jpg
reduced-isic/train_img/ISIC_0000026.jpg
reduced-isic/train_img/ISIC_0000027.jpg
reduced-isic/train_img/ISIC_0000023.jpg
reduced-isic/train_img/ISIC_0000022.jpg
reduced-isic/train_img/ISIC_0000029.jpg
reduced-isic/train_img/ISIC_0000013.jpg
reduced-isic/train_img/ISIC_0000003.jpg
reduced-isic/train_img/ISIC_0000024.jpg
reduced-isic/train_img/ISIC_0000015.jpg
reduced-isic/train_img/ISIC_0000001.jpg
reduced-isic/train_img/ISIC_0000032.jpg
reduced-isic/train_img/ISIC_0000004.jpg
reduced-isic/train_img/ISIC_0000028.jpg
reduced-isic/train_img/ISIC_0000031.jpg
reduced-isic/train_img/ISIC_0000000.jpg
reduced-isic/train_img/ISIC_0000030.jpg
reduced-isic/train_img/ISIC_0000007.jpg
reduced-isic/train_img/ISIC_0000025.jpg
reduced-isic/train_img/ISIC_0000016.jpg
reduced-isic/train_img/ISIC_0000017.jpg
reduced-isic/train_img/ISIC_0000011.jpg
reduced-isic/train_img/ISIC_0000012.jpg
reduced-isic/train_img/ISIC_0000006.jpg
reduced-isic/train_img/ISIC_0000020.jpg


In [12]:
!pip install dominate

In [13]:
!cp -r reduced-isic/train_edge train_label

In [15]:
# # Step 4: Train pix2pixHD and choose the best epoch using the Frechet Inception Distance
# # This uses pytorch instead of tensorflow, so we run these steps externally using the pix2pixHD code provided by nVidia
print('Training pix2pixhD with only training images... ')
start = time.time()
# # Train command: python train.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --dataroot pix2ultra/datasets/isic2018/ --nThreads 4 --display_winsize 256 --tf_log --no_instance --label_nc 2
!python pix2pixHD/train.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --dataroot ./ --nThreads 2 --display_winsize 256 --tf_log --no_instance --label_nc 2
# # FID command: python fid.py path/to/images path/to/other/images --gpu 0
end = time.time()
print('took {} seconds'.format(end-start))

Training pix2pixhD with only training images... 
------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: pix2ultra/checkpoints
continue_train: False
data_type: 32
dataroot: ./
debug: False
display_freq: 100
display_winsize: 256
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 2
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: pix2ultra
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: none
save_epoch_freq: 10
save_latest_freq: 1000


In [34]:
!ls pix2ultra/checkpoints/pix2ultra
# !zip -r pix2pix-check pix2ultra/checkpoints/

200_net_D.pth  iter.txt		 latest_net_G.pth  loss_log.txt  web
200_net_G.pth  latest_net_D.pth  logs		   opt.txt


In [40]:
# Step 5: Use the trained pix2pixHD model to generate synthetic ultrasound images from the synthetic edge diagrams
print(' all images... ')
start = time.time()
# Test command: python test.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --results_dir pix2ultra/results --how_many 3000 --dataroot pix2ultra/datasets/isic2018/ --display_winsize 256 --no_instance --label_nc 2
!python pix2pixHD/test.py --name pix2ultra --resize_or_crop none --checkpoints_dir pix2ultra/checkpoints --results_dir pix2ultra/results --how_many 30 --dataroot ./ --display_winsize 256 --no_instance --label_nc 2
end = time.time() 
print('took {} seconds'.format(end-start))

 all images... 
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: pix2ultra/checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./
display_winsize: 256
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 30
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 2
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: pix2ultra
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: pix2ultra/results
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset [Aligne